In [ ]:
# import stuff
import sys,os

import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors


#this works apparently only for savefig stuff
mpl.rcParams['figure.figsize']=(16.0,8.0)    #(6.0,4.0)
mpl.rcParams['font.size']=12             
   #10 

mpl.rcParams['savefig.dpi']=100             #72 
mpl.rcParams['figure.subplot.bottom']=.1    #.125

mpl.rcParams['legend.numpoints'] = 1

plt.rc('font', family='serif')
plt.rc('text', usetex=True)

#inline Shit

%matplotlib inline
%config InlineBackend.figure_format='svg'
%config InlineBackend.rc = {'figure.facecolor': 'white', 'figure.subplot.bottom': 0.125, 'figure.edgecolor': 'white', 'savefig.dpi': 300, 'figure.figsize': (12.0, 8.0), 'font.size': 10}

#GUi shit
%matplotlib tk

mpl.get_configdir()

%load_ext autoreload
%autoreload 2
# Import General Packages from me
from Tools.Parsers import *
from Tools.BoundingBox import *
from transforms3d.quaternions import *


In [ ]:
# plot dynState
from mpldatacursor import datacursor

def plotData(dynState):
    plt.close('all')
    
    
    fig, (ax0,ax1,ax2) = plt.subplots(nrows = 3);
    n = len(dynState.bodies)
    jet = plt.get_cmap('jet') 
    cNorm  = colors.Normalize(vmin=0, vmax=n)
    scalarMap = cm.ScalarMappable(norm=cNorm, cmap=jet)
    
    
    # Position
    
    for i,body in dynState.bodies.items():
        linex, = ax0.plot(dynState.t, body[0][:,0],'.-', color= scalarMap.to_rgba(i), label='x , b: ' + str(i));
        liney, = ax0.plot(dynState.t, body[0][:,1],'.-', color= scalarMap.to_rgba(i), label='y , b: ' + str(i));
        linez, = ax0.plot(dynState.t, body[0][:,2],'.-', color= scalarMap.to_rgba(i), label='z , b: ' + str(i));
        datacursor([linex,liney,linez],draggable=True)
        ax1.plot(dynState.t, body[0][:,3:7],'.-', color= scalarMap.to_rgba(i))
        ax2.plot(dynState.t, body[2]['rank'][:,0],'.-', color= scalarMap.to_rgba(i));
        
    
    
    ax0.grid(True)
    ax0.set_title('Position');
    ax0.set_ylabel('\mathbf{r}_S(t)');
    ax0.set_ymargin(0.2)
    ax0.autoscale(True,'both',False)
    
    ax1.grid(True)
    ax1.set_title('Quaternion');
    ax1.set_ylabel('\mathbf{a}_{KI}(t)');
    ax1.set_ymargin(0.2)
    ax1.autoscale(True,'both',False)
    
    ax2.grid(True)
    ax2.set_title('Rank');
    ax2.set_ylabel('rank number');
    ax2.set_ymargin(0.2)
    ax2.autoscale(True,'both',False)
    
    
    # Velocity
    fig2, (ax20,ax21) = plt.subplots(nrows = 2);
    
    
    
    for i,body in dynState.bodies.items():               
        ax20.plot(dynState.t, body[1][:,0:3], '.-' , color= scalarMap.to_rgba(i));
        ax21.plot(dynState.t, body[1][:,3:6], '.-' , color= scalarMap.to_rgba(i));
    
    ax20.grid(True)
    ax20.set_title('Velocity');
    ax20.set_ylabel('\mathbf{v}_S(t)');
    ax20.set_xmargin(0.0)
    ax20.set_ymargin(0.1)
    
    ax21.grid(True)
    ax21.set_title('Angular Velocity');
    ax21.set_ylabel('\mathbf{\omega}_{KI}(t)');
    ax21.set_xmargin(0.0)
    ax21.set_ymargin(0.1)
    return ( (fig,(ax0,ax1,ax2)) , (fig2, (ax20,ax21) )  );

def plotData3D(dynState):
    pass
        

Plot SimData.dat for MPI
---

In [ ]:
# plot SimData MPI

import csv, re, os, copy
plt.close('all')

class PlotSimDataMPI:
    
    def __init__(self,localFolderPath, simDataRelPath="./SimData.dat",  regExProcessFolder=".*ProcessMPI_(\d*).*", simDataGuiPath=""):
        self.simDataGuiPath=simDataGuiPath;
        self.simDataRelPath=simDataRelPath;
        self.regExProcessFolder=regExProcessFolder
        self.localFolderPath=localFolderPath;
        
    def plot(self, oneWindow=False):
        # Plot stuff
        self.oneWindow = oneWindow;
        
        self.generatePlotsAndAxes()
        
        self.loadData(self.localFolderPath)
        self.plotSimDataMPI(self.simDataList)
        
        if(self.simDataGuiPath):
            self.plotComparisionGUI(simDataGuiPath)
        
        
        
        # add legends
        self.axdict['nContacts'].legend()
        self.axdict['nSplitBodyNodes'].legend()
        
        #Create custom legend for remote and local contacts
        options = self.generateOptions(0,self.scalarMap)
        localContactsArtist = plt.Line2D((0,1),(0,0), linestyle='' ,marker='s',**options['optMarker2'])
        remoteContactsArtist = plt.Line2D((0,1),(0,0),linestyle='', marker='o',**options['optMarker3'])
        
        #Create legend from custom artist/label lists
        self.axdict['nContactsAllRL'].legend([localContactsArtist,remoteContactsArtist],['Local', 'Remote'])
        
        
        # adjust margin
        for t in self.axarr:
            t.autoscale(True,tight=True)
            t.autoscale_view(True,True,True)
            t.margins(0.1)
        
        for f in self.figarr:
             f.canvas.draw()
             f.tight_layout();
                
        
    def generatePlotsAndAxes(self):
        
        self.axarr = [];
        self.figarr = [];
        
        if not self.oneWindow :
            
            for i in range(0,9):
                f = plt.figure(i);
                self.figarr.append(f);
                ax = f.add_subplot(111);
                self.axarr.append(ax);
                
        else:
            fig, axarr = plt.subplots(3,3, sharex=True)
            self.figarr.append(fig);
            self.axarr.extend(list(axarr.flatten()));
 
        self.axdict = dict();
        
        self.axdict['TotalStateEnergy']= self.axarr[0];
        self.axdict['TotalStateEnergy'].set_title('Energy')
        #axdict['TotalStateEnergy'].set_xlabel('time [s]');
        self.axdict['TotalStateEnergy'].set_ylabel('T+V [J]');
        
        self.axdict['nGlobalIterations'] = self.axarr[1];
        self.axdict['nGlobalIterations'].set_title('Iterations Prox')
        #axdict['nGlobalIterations'].set_xlabel('time [s]');
        self.axdict['nGlobalIterations'].set_ylabel('\# iterations');
        
        self.axdict['Converged'] = self.axarr[2];
        self.axdict['Converged'].set_title('Prox Converged')
        #axdict['Converged'].set_xlabel('time [s]');
        self.axdict['Converged'].set_ylabel('converged [0,1]');
        
        self.axdict['nContacts'] = self.axarr[3];
        self.axdict['nContacts'].set_title('Number of Contacts')
        #axdict['nContacts'].set_xlabel('time [s]');
        self.axdict['nContacts'].set_ylabel('\# contacts');
        
        self.axdict['nContactsAll'] = self.axarr[4];
        self.axdict['nContactsAll'].set_title('Number of Contacts')
        #axdict['nContacts'].set_xlabel('time [s]');
        self.axdict['nContactsAll'].set_ylabel('\# contacts');
        
        self.axdict['nContactsAllRL'] = self.axarr[5];
        self.axdict['nContactsAllRL'].set_title('Number of Remote and Local Contacts')
        #axdict['nContacts'].set_xlabel('time [s]');
        self.axdict['nContactsAllRL'].set_ylabel('\# contacts');
        
        
        self.axdict['TimeStepTime'] = self.axarr[6];
        self.axdict['TimeStepTime'].set_title('Time-Step time')
        #axdict['TimeStepTime'].set_xlabel('time [s]');
        self.axdict['TimeStepTime'].set_ylabel('[s]');
        
        self.axdict['MaxOverlap'] = self.axarr[7];        
        self.axdict['MaxOverlap'].set_title('MaxOverlap')
        #axdict['MaxOverlap'].set_xlabel('time [s]');
        self.axdict['MaxOverlap'].set_ylabel('[m]');
        
        self.axdict['nSplitBodyNodes'] = self.axarr[8];
        self.axdict['nSplitBodyNodes'].set_title('Number of SplitNodes')
        self.axdict['nSplitBodyNodes'].set_xlabel('time [s]');
        self.axdict['nSplitBodyNodes'].set_ylabel('\# SplitBodies');
        
        
         # Total plots:
        
        f = plt.figure(9);
        ax = f.add_subplot(111);
        self.axdict['maxTimings'] = ax;
        ax.set_title('Maximum Timings of all Processes');
        ax.set_xlabel('time [s]');
        ax.set_ylabel('cpu time [s]');
        self.figarr.append(f);
        self.axarr.append(ax);
        
        
        
        for a in self.axdict.values():
            a.grid(True)
        
    def loadData(self,folderPath):
        processNumbers = [];
        # Looop through process folder
        simDataFiles = dict();
        for d in os.listdir(folderPath):
            p = os.path.normpath(os.path.join(folderPath,d));
            if os.path.isdir(p):
                res = re.match(self.regExProcessFolder,p)
                if(res):
                    #print("Adding folder: ", p)
                    processNumber = res.group(1);
                    processNumbers.append(processNumber)
                    simDataFiles[int(processNumber)] = os.path.normpath(os.path.join(p, self.simDataRelPath));
        
        print("SimDataFiles from Processes: " + str(processNumbers) )
        
        self.simDataList =  self.combineSimData(simDataFiles);  # first is the combine, all others are the individual 
     
    def generateColumnNames(self,simDataFile):
        # open and read header
        with open(simDataFile, 'r') as f:
            reader = csv.reader(f, delimiter='\t', quoting=csv.QUOTE_NONE)
            header = next(reader)
        #print("Header is " , header)
        # Replace "#"
        header[0] = header[0].replace('#',"")
        header = [s.strip() for s in header]
        
        columnNames =[]
        for i,s in enumerate(header):
            #print(i,s)
            res = re.match('^([\w#]*\s?)*\w+',s)
            if(res):
                columnNames.append(res.group(0))
            else:
                raise NameError("No match in string " + str(s))
                
        return columnNames;
    
    def plotSimDataMPI(self,simDataList):
        
        self.scalarMap = self.generateColorMap(len(simDataList));
        self.nProcesses = len(simDataList);
        for processNr, SimData in simDataList.items():
            optionsDict = self.generateOptions(processNr, self.scalarMap);
            if( processNr == 'Total'):
                self.plotSimDataTotal(SimData,optionsDict);
            elif( processNr == 'Average'):
                self.plotSimDataAverage(SimData,optionsDict);
            else:
                self.plotSimDataProcess(SimData,processNr,optionsDict) 
    
    def plotSimDataAverage(self,SimDataAvg, optionsDict=dict()):
        self.axdict['MaxOverlap'].plot(SimDataAvg['SimulationTime'],SimDataAvg['MaxOverlap'],'k-', lw=2);    
    
    def plotSimDataProcess(self,SimData, processNr, optionsDict=dict()):
            #Energy
            #axdict['TotalStateEnergy'].plot(SimData['SimulationTime'],SimData['TotalStateEnergy']);
            
            #Iterations
            self.axdict['nGlobalIterations'].plot(SimData['SimulationTime'],SimData['nGlobalIterations'],'-', **optionsDict['optLine']); 
            #Convergence
            self.axdict['Converged'].plot(SimData['SimulationTime'],SimData['Converged'],'o', **optionsDict['optMarker'] );
            
            #Contacts Number All
            self.axdict['nContactsAll'].plot(SimData['SimulationTime'],SimData['nContacts'],'o', **optionsDict['optMarker'] );
            
            #Contacts Number remote Local
            self.axdict['nContactsAllRL'].plot(SimData['SimulationTime'],SimData['nContactsLocal'],'s',**optionsDict['optMarker2'] );
            self.axdict['nContactsAllRL'].plot(SimData['SimulationTime'],SimData['nContactsRemote'],'o',**optionsDict['optMarker3'] );
            
            #SplitNodes Number
            self.axdict['nSplitBodyNodes'].plot(SimData['SimulationTime'],SimData['nSplitBodyNodes'],'o', **optionsDict['optMarker'] );
            
            #TimeStep Time
            self.axdict['TimeStepTime'].plot(SimData['SimulationTime'],SimData['TimeStepTime'],'-', **optionsDict['optLine']);
            
            #MaxOverlap
            self.axdict['MaxOverlap'].plot(SimData['SimulationTime'],SimData['MaxOverlap'],'-', **optionsDict['optLine']);
            
    def plotSimDataTotal(self, SimDataT, optionsDict=dict()):
        self.axdict['nContacts'].plot(SimDataT['SimulationTime'],SimDataT['nContacts'],'o',**optionsDict['optMarker'] );
        self.axdict['TotalStateEnergy'].plot(SimDataT['SimulationTime'],SimDataT['TotalStateEnergy'],'-', **optionsDict['optLine']);
        self.axdict['TotalStateEnergy'].set_xlim((0,SimDataT['SimulationTime'][-1]))
        
        self.axdict['maxTimings'].plot(SimDataT['SimulationTime'] , SimDataT['TimeStepTime'], 'k-', label="total cpu time");
        
        polys = self.axdict['maxTimings'].stackplot(SimDataT['SimulationTime'] ,  
                                            SimDataT['BodyCommunicationTime'], 
                                            SimDataT['CollisionTime'] , 
                                            SimDataT['InclusionTime']);
        legendProxies = []
        for poly in polys:
            legendProxies.append(plt.Rectangle((0, 0), 1, 1, fc=poly.get_facecolor()[0]))
        self.axdict['maxTimings'].legend(legendProxies, ['body communication','collision time','inclusion time' ])
        
        self.axdict['MaxOverlap'].plot(SimDataT['SimulationTime'],SimDataT['MaxOverlap'],'k-', lw=2);
        
    def generateColorMap(self,l):
        jet = colorMap = plt.get_cmap('jet') 
        cNorm  = colors.Normalize(vmin=0, vmax=l)
        scalarMap = cm.ScalarMappable(norm=cNorm, cmap=colorMap)
        return scalarMap;
    
    def generateOptions(self,processNr, scalarMap):
        
        if(processNr == 'Total'):
            processNr = 0
            label = 'MPITot'
            colorVal = scalarMap.to_rgba(0);
        elif( processNr == 'Average'):
            return;
        else:
            label = 'MPI%d' % processNr
            colorVal = scalarMap.to_rgba(processNr+1);
            
        optLine = {      'linewidth':1, 
                         'color':colorVal, 'label' : label
                  }
        
        optMarker = {    'markersize':1,#(processNr+1)*2+6, 
                         'markeredgecolor' : colorVal, 
                         'markerfacecolor' : 'none', 
                         'markeredgewidth': 2,
                         'label' : label
                    }
        optMarker2 = {   'markersize':1,#(processNr+1)*3+10,
                         'markeredgecolor' : colorVal, 
                         'markerfacecolor': 'none', 
                         'markeredgewidth': 2
                     }
        optMarker3 = {   'markersize':1,#(processNr+1)*3+5, 
                         'markeredgecolor' : colorVal, 
                         'markerfacecolor': 'none', 
                         'markeredgewidth': 2,
                     }
        optStep =  { 'markersize':3, 
                     'markeredgecolor':colorVal, 
                     'markerfacecolor':'none', 
                     'markeredgewidth':(processNr+1)*4,
                     'where' : 'post',
                     'label' : label
                    }
        
        return {'optLine': optLine, 
                'optMarker': optMarker, 
                'optStep': optStep, 
                'optMarker2': optMarker2,
                'optMarker3': optMarker3
                }

        
    def combineSimData(self,simDataFiles):
        #load first file
        simDataList = dict();
        simDataFile = simDataFiles.pop(0); # first process 
        columnNames = self.generateColumnNames(simDataFile)
        dt = [(s,np.float64) for s in columnNames]
        print("dtype: " +str(dt))
        print("columnNames: " + str(columnNames) )
        # load structured unit        
        SimData = np.loadtxt(simDataFile,dtype=dt, delimiter="\t", comments='#')
        simDataList[0]=SimData;
        print("SimulationTime:" , simDataList[0]['SimulationTime'])
        
        # add the other processes
        for processNr, simDataFile in simDataFiles.items():
            cNT = self.generateColumnNames(simDataFile)
            if(cNT != columnNames):
                raise NameError(str(cNT) + " is not the same as " + str(columnNames))
            # load structured unit        
            simDataList[processNr] = np.loadtxt(simDataFile,dtype=dt, delimiter="\t", comments='#');
            
        
        #determine common number of rows for SimDataTotal (not all processes might have written equal amount of rows... -> shutdown)
        minRows = np.shape(simDataList[0])[0];
        for simData in simDataList.values():
            minRows = min(minRows, np.shape(simData)[0]);
        print("minRows: " + str(minRows));
        
        print("Structured Array Shape Process 0:" , np.shape(simDataList[0]))
        
        #make dictionary for total maximal values
        SimDataTotal = np.zeros((minRows,),dtype= np.result_type(simDataList[0]));
        SimDataTotal['SimulationTime'] = simDataList[0]['SimulationTime'][0:minRows];
        
        
        # make dictionary for average values 
        SimDataAvg = {}
        SimDataAvg['SimulationTime'] = np.array(simDataList[0]['SimulationTime'][0:minRows]);
        SimDataAvg['MaxOverlap'] = [];
        
        for simData in simDataList.values():
            SimDataTotal['nContacts'] += simData['nContacts'][0:minRows]
            SimDataTotal['TotalStateEnergy'] += simData['TotalStateEnergy'][0:minRows] 
            
            SimDataTotal['TimeStepTime'] = np.maximum( SimDataTotal['TimeStepTime'], simData['TimeStepTime'][0:minRows] )
            SimDataTotal['CollisionTime'] = np.maximum( SimDataTotal['CollisionTime'], simData['CollisionTime'][0:minRows] )
            SimDataTotal['InclusionTime'] = np.maximum( SimDataTotal['InclusionTime'], simData['InclusionTime'][0:minRows] )
            SimDataTotal['BodyCommunicationTime'] = np.maximum( SimDataTotal['BodyCommunicationTime'], simData['BodyCommunicationTime'][0:minRows] )
            SimDataTotal['AvgIterTime'] = np.maximum( SimDataTotal['AvgIterTime'], simData['AvgIterTime'][0:minRows] )
            SimDataTotal['MaxOverlap'] = np.maximum( SimDataTotal['MaxOverlap'], simData['MaxOverlap'][0:minRows] )
            SimDataTotal['TotalTimeProx'] = np.maximum( SimDataTotal['TotalTimeProx'], simData['TotalTimeProx'][0:minRows] )
            SimDataTotal['IterTimeProx'] = np.maximum( SimDataTotal['IterTimeProx'], simData['IterTimeProx'][0:minRows] )
            
            
            SimDataAvg['MaxOverlap'].append(simData['MaxOverlap'][0:minRows])
        
        #average all entries in 
        for key,value in SimDataAvg.items():
            if(key!="SimulationTime"):
                fd = np.array(value);
                SimDataAvg[key] = np.average(fd,axis=0);
        
        simDataList['Average']=SimDataAvg;
        simDataList['Total']=SimDataTotal;
        
        return simDataList;
    
    
    def plotComparisionGUI(self,simDataFile):
        #compare to other SimData
        columnNames = self.generateColumnNames(simDataFile)
        dt = [(s,np.float64) for s in columnNames]
        # load structured unit        
        SimDataCompare = np.loadtxt(simDataFile,dtype=dt, delimiter="\t", comments='#')
        self.plotSimData(SimDataCompare)
        #SimDataCompare['nContacts'] == simDataList['Total']['nContacts']
        
#     def plotSimData(self,SimData):
        
#         optionsDict = { 'optLine' : {  'linewidth':1, 
#                                        'color':'k',
#                                        'label' : 'GUI'
#                                     } ,
            
#                         'optMarker': {   'markersize':7, 
#                                          'markeredgecolor':'none',
#                                          'markerfacecolor':'k',
#                                          'label' : 'GUI'
#                                      },
#                         'optStep' :  {   'markersize':7, 
#                                          'markeredgecolor':'none',
#                                          'markerfacecolor':'k',
#                                          'label' : 'GUI',
#                                          'where' : 'post'
#                              }
#                       }
#         #Energy
#         self.axdict['TotalStateEnergy'].plot(SimData['SimulationTime'],SimData['TotalStateEnergy'],'-',**optionsDict['optLine']);
        
#         #Iterations
#         self.axdict['nGlobalIterations'].plot(SimData['SimulationTime'],SimData['nGlobalIterations'],'-', **optionsDict['optLine']); 
#         #Convergence
#         self.axdict['Converged'].plot(SimData['SimulationTime'],SimData['Converged'],'.', **optionsDict['optMarker']);
        
#         #Contacts Number
#         self.axdict['nContacts'].plot(SimData['SimulationTime'],SimData['nContacts'],'.', **optionsDict['optMarker']);
    
#         #TimeStep Time
#         #axdict['TimeStepTime'].plot(SimData['SimulationTime'],SimData['TimeStepTime'],'-', **optionsLinePlot);
        
#         #MaxOverlap
#         axdict['MaxOverlap'].plot(SimData['SimulationTime'],SimDataTotal['MaxOverlap'],'k-', lw=3);
          
#end class

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# x = np.linspace(0,10,31)

# fig = plt.figure()
# ax = fig.add_subplot(1,1,1)

# #Create custom artists
# simArtist = plt.Line2D((0,1),(0,0), color='k', marker='o', linestyle='')
# anyArtist = plt.Line2D((0,1),(0,0), color='k')

# #Create legend from custom artist/label lists
# ax.legend([simArtist,anyArtist],
#           ['Simulation', 'Analytic'])

# plt.show()

In [ ]:

bodyRange = [0,500]
timeRange = [0,5]

#For single file which contains multiple bodies
#dynState = ReadInSimFile("./SimFiles/SimDataRECORDGUI_0/SimulationState.sim",[0,10],[0,10],False)
#For multiple file which contains 1 body!
#dynState = ReadInSimFileMulti("./SimFiles/SimDataRECORD_0/", [0,10] , [0,10], "SimDataBody-1-(\d*)\.sim" ,True)
#print(dynState)

folderNumber= 0
filePath = os.path.join("../GlobalFolder/SimFiles/" , "SimDataRECORDMPI_%d" % folderNumber, "./SimState.sim");
#filePath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-DriftC/first/SimStateRenderTest.sim"

dynState = SimFileReader.ReadInSimFile( filePath ,timeRange,bodyRange,False)

plotData(dynState)
res = plotData3D(dynState);


In [ ]:
localFolderPath = "../LocalFolder/"
#localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-DriftC/first/"

simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI);
p.plot()

Plot MPI Topology
---

In [ ]:
# makeSymlinks
import glob2;


def makeLinks(fileGlobPattern, filePathRegex, subPattern, outputFolder, 
              counters = [(0,lambda x: os.path.basename(x))] , hardLink=False):
    """ Globs all files with the pattern  fileGlobPattern
        and builds for each found file a new symlink where the
        file name is determined by match/substituting the filePath 
        with the regex: filePathRegex and substitution pattern: subPattern which produces the new file name
        counter is a list of tuples each consisting of (start, key lambda) which defines the start of a 
        counter and a sort lambda which takes a filepath and returns a sort key for the sort function.
        all associated counters are then replaced (formatted) in the new file name , substituted by the subPattern.
        
        Then, it saves a new symlink in the outputFolder for each of these new file names.
        Example:
        makeSymlinkSequence("/media/zfmgpu/Data1/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-RunDownTrees"+
                     "/*/ProcessMPI_0/SimDataRECORDMPI_0/TopologyInfo_*.xml", 
                     r".*Trees/(\d)*/.*_(\d)*.*", 
                     r"TopologyInfo_\1-\2",
                     "/media/zfmgpu/Data1/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-RunDownTrees/AllTopologies/")
    """
    reg = re.compile(filePathRegex)
    files = glob2.glob(fileGlobPattern);
    newFileNames = [  reg.sub(subPattern,f) for f in files]
    files = zip(files,newFileNames)


    for counter in counters:
        count = counter[0] # the start of this counter
        # sort files alphabetically
        # hand over the sort key lambda of this counter
        files = sorted(files, key = lambda x: counter[1](x[0]) ) 
        # assign counter alphabetically
        for i,f in enumerate(files):
            files[i] = f + (count,)
            count += 1
        # sort files for modifcation times
        # TODO (assign counter)
    
    os.makedirs(outputFolder,exist_ok=True);
    print("Make symlinks in folder: ", outputFolder)
    # make symlinks to all TopologyInfos in outputFolder
    for tu in files:
        f = tu[0]
        s = tu[1]
        simLinkFile = s.format(*tu[2:]);
        simLinkPath = os.path.join(outputFolder,simLinkFile) 
        print("Link: " + os.path.basename(f) + " --- to ---> " + simLinkPath)
        try:
            if hardLink:
                os.link(f, simLinkPath );
            else:
                os.symlink(f, simLinkPath );
        except FileExistsError:
            continue;
    


In [ ]:
# plot Topology MPI
import re, os
import numpy as np
import math
from mpl_toolkits.mplot3d import Axes3D
import visvis as vv
from visvis import Point, Pointset

import Tools.Parsers.TopologyInfo as tpI

def plotCube(ax,
             minP = np.array([-1.0,-1.0,-1.0]), 
             maxP=np.array([1.0,1.0,1.0]),
             trans= np.array([0.0,0.0,0.0]),
             rotMatrix=np.diag([1,1,1]), subdivs=None, opts={}, optsDivs={}):
    from itertools import product, combinations
    r = [-1, 1]

    centerPos = (maxP + minP)/2.0;
    extent = (maxP - minP)/2.0;

    points = np.array([(-1, -1, -1),
                     (-1, -1, 1),
                     (-1, 1, -1),
                     (-1, 1, 1),
                     (1, -1, -1),
                     (1, -1, 1),
                     (1, 1, -1),
                     (1, 1, 1)]);

    for s, e in combinations(points, 2):
        if np.sum(np.abs(s-e)) == r[1]-r[0]: # no diagonal lines
            p1 = np.array(s,dtype=float); p2 = np.array(e,dtype=float);
            #scale points
            p1*=extent; p2*=extent;
            #rotate and translate points
            p1 = rotMatrix.dot(p1 + centerPos) + trans;
            p2 = rotMatrix.dot(p2+centerPos) + trans;
            vv.plot(*zip(p1,p2), axes=ax,**opts);
    
    if subdivs is not None:
        pp = list();
        dxyz = (maxP - minP) / subdivs;

        for k in range(0,int(subdivs[1])+1):
            for l in range(0,int(subdivs[2])+1):
                pp.extend( ([ minP[0], minP[1] + k*dxyz[1],  minP[2] + l*dxyz[2] ],
                           [ maxP[0], minP[1] + k*dxyz[1],  minP[2] + l*dxyz[2] ]) 
                );
                
        for k in range(0,int(subdivs[2])+1):
            for l in range(0,int(subdivs[0])+1):
                pp.extend( ([ minP[0]+ l*dxyz[0],  minP[1],  minP[2] + k*dxyz[2] ],
                            [ minP[0]+ l*dxyz[0],  maxP[1],  minP[2] + k*dxyz[2] ] )
                         );
        
        for k in range(0,int(subdivs[0])+1):
            for l in range(0,int(subdivs[1])+1):
                pp.extend( ([ minP[0]+k*dxyz[0],   minP[1]+l*dxyz[1],  minP[2] ],
                            [  minP[0]+k*dxyz[0],   minP[1]+l*dxyz[1],  maxP[2]])
                          ); 
        p = np.array(pp);
        
        #Transform all subdivs points into the I frame
#         print(rotMatrix)
#         print(type(p))
        for point in p:
            point[...] = rotMatrix.dot(point) + trans;
        # print(p)
        vv.plot(p.T[0],p.T[1],p.T[2],axes=ax,**optsDivs)

def plotCubeMesh(ax,
                 minP = np.array([-1.0,-1.0,-1.0]), 
                 maxP =np.array([1.0,1.0,1.0]),
                 trans= np.array([0.0,0.0,0.0]),
                 rotQuat=np.array([1.0,0.0,0.0,0.0]),  opts={} ):
                 
        centerPos = (maxP + minP)/2.0 + trans;
        extent = (maxP - minP);
        axis,angle = quat2axangle(rotQuat);
        print(angle,axis)
        print(centerPos,extent)
        c = vv.solidBox(axes=ax, 
                    translation=(centerPos[0],centerPos[1],centerPos[2]),
                    scaling=(extent[0],extent[1],extent[2]),direction=(axis[0],axis[1],axis[2]),rotation=angle/math.pi*180.0)
        c.faceShading = None;
        c.edgeShading = 'plain';
        
        if 'lc' in opts:
            c.edgeColor = opts['lc'];
        

        

def generateLinesCube(minP = np.array([-1.0,-1.0,-1.0]), 
              maxP = np.array([1.0,1.0,1.0]), 
              rotMatrix = None,
              subdivs=None, opts={}, optsDivs={}):
    
    centerPos = (maxP + minP)/2.0;
    extent = (maxP - minP);
    
    nLines = 12;
    # for each subdiv a quad is (4 lines) is drawn
    if subdivs:
       nLines += (subdivs[0]+1)*(subdivs[1]+1) + (subdivs[1]+1)*(subdivs[2]+1) + (subdivs[2]+1)*(subdivs[0]+1) - 4*3;

    lines = np.zeros((nLines*2,3), dtype=np.float32);
    
    # plot lines
    lines[0:24,0:3] = 0.5* np.array([ (-1, -1, -1),
                                    (1, -1, -1),
                                    #
                                    (1, -1, -1),
                                    (1, 1, -1),
                                    #
                                    (1, 1, -1),
                                    (-1, 1, -1), 
                                    #
                                    (-1, 1, -1),
                                    (-1, -1, -1), # end of first quad
                                    #
                                    (-1, -1, 1),
                                    (1, -1, 1),
                                    #
                                    (1, -1, 1),
                                    (1, 1, 1),
                                    #
                                    (1, 1, 1),
                                    (-1, 1, 1), 
                                    #
                                    (-1, 1, 1),
                                    (-1, -1, 1), # end of second quad
                                    #
                                    (-1, -1, -1),
                                    (-1, -1, 1),
                                    #
                                    (1, -1, -1),
                                    (1, -1, 1),
                                    #
                                    (-1, 1, -1),
                                    (-1, 1, 1), 
                                    #
                                    (1, 1, -1),
                                    (1, 1, 1)]); # end of vertical lines
    
    
    # add subdivs if necessary
    counter = 12*2; # start after the cube lines
    if subdivs is not None:
        dxyz = np.array([1.0 / subdivs[0],1.0 / subdivs[1],1.0 / subdivs[2]]);
        
        # y
        # |    +   +   +   
        # | *  *   *   *   *
        # | *  *   *   *   *
        # |    +   +   +  
        # ----------------> x
        #  * come first (, then the +
        
        # make x lines inside (*)
        for k in range(1,int(subdivs[1])):
            for l in range(0,int(subdivs[2])+1):
                lines[counter:counter+2,0:3] = np.array([ (-0.5, -0.5 + k*dxyz[1],  -0.5 + l*dxyz[2]),
                                                          (0.5, -0.5 + k*dxyz[1],  -0.5 + l*dxyz[2])  ]);
                counter+=2;
        # make the x lines (+)
        for k in (0,subdivs[1]):
            for l in range(1,int(subdivs[2])):
                lines[counter:counter+2,0:3] = np.array([ (-0.5, -0.5 + k*dxyz[1],  -0.5 + l*dxyz[2]),
                                                          (0.5, -0.5 + k*dxyz[1],  -0.5 + l*dxyz[2])  ]);
                counter+=2;
        
        
        # make y lines inside (*)     
        for k in range(1,int(subdivs[2])):
            for l in range(0,int(subdivs[0])+1):
                lines[counter:counter+2,0:3] = np.array([ (-0.5+ l*dxyz[0],  -0.5,  -0.5 + k*dxyz[2]),
                                                          (-0.5+ l*dxyz[0],  0.5,  -0.5 + k*dxyz[2])  ]);
                counter+=2;
        # make the y lines (+)
        for k in (0,subdivs[2]):
            for l in range(1,int(subdivs[0])):
                lines[counter:counter+2,0:3] = np.array([ (-0.5+ l*dxyz[0],  -0.5,  -0.5 + k*dxyz[2]),
                                                          (-0.5+ l*dxyz[0],  0.5,  -0.5 + k*dxyz[2])  ]);
                counter+=2;
                
        # make z lines inside
        for k in range(1,int(subdivs[0])):
            for l in range(0,int(subdivs[1])+1):
                lines[counter:counter+2,0:3] = np.array([ (-0.5+k*dxyz[0],  -0.5+l*dxyz[1],  -0.5),
                                                          (-0.5+k*dxyz[0],  -0.5+l*dxyz[1],  0.5)    ]); 
                counter+=2;
        # make the z lines (+)
        for k in (0,subdivs[0]):
            for l in range(1,int(subdivs[1])):
                lines[counter:counter+2,0:3] = np.array([ (-0.5+k*dxyz[0],  -0.5+l*dxyz[1],  -0.5),
                                                          (-0.5+k*dxyz[0],  -0.5+l*dxyz[1],  0.5)    ]); 
                counter+=2;
        
        
        lines[...] *= extent;
        if rotMatrix is not None:
            lines = np.dot(lines,rotMatrix.T); # [p1.T; p2.T] * R.transpose
        lines[...] += centerPos;    
    return lines;
        
def plotAxis(ax,centerPos,A_IK,plotAxisScale=1):
        for i,c in zip([0,1,2],['r','g','b']):
            I_eK_i = A_IK[:,i];
            lines = list(zip(centerPos,plotAxisScale*I_eK_i+centerPos))
            #v = Arrow3D(*lines, mutation_scale=50, lw=1, arrowstyle="-|>", color=c);
            vv.plot(*lines, lc=c,axes=ax)

            

        

class PlotTopologyMPI:
    def __init__(self,folderPath, filePathRegex, index):
        self.folderPath=folderPath;
        self.filePathRegex=filePathRegex
        self.fileIndex = index;
        self.loadData(self.folderPath, self.filePathRegex, self.fileIndex);
        
    def axisEqual3D(self,ax):
        extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
        sz = extents[:,1] - extents[:,0]
        centers = np.mean(extents, axis=1)
        maxsize = max(abs(sz))
        r = maxsize/2
        for ctr, dim in zip(centers, 'xyz'):
            getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)    
            
    def makeWindow(self,i):
        fig = vv.figure();
        fig.title = "TopoInfo " + i;
        ax = vv.gca();
        ax.cameraType = '3d'
        ax.daspectAuto = False
        vv.xlabel("x");
        vv.ylabel("y");
        vv.zlabel("z");
        return (fig,ax);           
    
    def plot(self, index , ax = None, oneWindow = False, plotPoints = False, plotSubDivs = False):
        # Plot stuff
        
        
        if oneWindow:
            fig,ax = self.makeWindow("all");
        
        if not index:
            index = self.topoInfos.keys();
            
        # Plot TopoInfo for index = 0 
        for i,t in self.topoInfos.items():
            #print("Plot TopoInfo %i" % i)
            if not oneWindow:
                fig,ax = self.makeWindow(str(i));
                
            c = self.colorMap.to_rgba(i)[0:3]
            opts={ 'grid':{'lc':c,'mc':c} , 'points':{'lc':'','ls':'','mc':c,'ms':'.','mw':2}, 'subdivs' :{'ls':'+'} }
            self.plotTopoInfo(i,self.topoInfos[i],ax,plotPoints,plotSubDivs,opts);
        
        
        app = vv.use();
        app.Run();
        
    
        
    def generatePlotsAndAxes(self):
        pass;
        
    def loadData(self,folderPath, filePathRegex, fileIndex):
        if not fileIndex:
            loadAll=True;
        else:
            loadAll=False;
            
        # Looop through process folder
        self.topoInfos = dict();
        files = [ os.path.join(folderPath,f) for f in os.listdir(folderPath) if os.path.isfile( os.path.join(folderPath,f) ) ]
        for f in files:
            res = re.match(filePathRegex,f)
            if(res):
                print("Parsing file: ", f)
                fileNr = res.group(1);
                if not loadAll:
                    load = int(fileNr) in fileIndex;
                else:
                    load = True;
                if load:
                    t = TopologyXMLParser.parseTopologyInfoXML(f,False);
                    self.topoInfos[int(fileNr)] = t;
        
        print("TopoInfos: " + str(self.topoInfos) )
        
        # init colormap
        self.colorMap = self.generateColorMap(len(self.topoInfos))
    
    def plotTopoInfo(self,i,topoInfo,ax,plotPoints,plotSubDivs,plotOpts):
        
        if( isinstance(topoInfo.topo , tpI.Grid) ):
            
            # Plotting GRID
            
            if(plotSubDivs):
                subdivs = topoInfo.grid.processDim;
            else:
                subdivs = None;

            g = topoInfo.topo;
            if(g.aligned):
                plotCube(ax, g.aabb.min, g.aabb.max , subdivs=subdivs ,opts=plotOpts['grid'], optsDivs=plotOpts['subdivs']);

                #plot label
                p = g.aabb.min;

                vv.Text(ax, x=p[0], y=p[1], z=p[2], text="Topo %i" % i);
            else:
                plotCube(ax, g.aabb.min, g.aabb.max , rotMatrix = g.A_IK, subdivs=subdivs, opts=plotOpts['grid'], optsDivs=plotOpts['subdivs']);
                #plot label
                p = g.A_IK.dot(g.aabb.min);
                vv.Text(ax, x=p[0], y=p[1], z=p[2], text="Topo %i" % i);

            if(plotPoints and np.size(topoInfo.points)):
                pp = Pointset(topoInfo.points);
                vv.plot(topoInfo.points.T[0],topoInfo.points.T[1],topoInfo.points.T[2], axes=ax, **plotOpts['points']);
        
        
        elif( isinstance(topoInfo.topo , tpI.KdTree) ):
            
            # Plotting KDTREE
            kdTree = topoInfo.topo;
            for level, aabbs in kdTree.aabbTree.items():
                print(level,np.shape(aabbs))
                for aabb in aabbs:
                    #print(aabb)
                    #plotCube(ax, aabb['min'], aabb['max'] , rotMatrix = kdTree.A_IK, subdivs=None, opts=plotOpts['grid'], optsDivs=plotOpts['subdivs']);
                    plotCubeMesh(ax, aabb['min'], aabb['max'] , rotQuat = kdTree.q_KI, opts=plotOpts['grid']);
                
            
            
        else:
            raise NameError("TopoInfo has unsupported type: " + str(type(topoInfo)) + " for plotting!")
    
    def generateColorMap(self,l):
        jet = colorMap = plt.get_cmap('jet') 
        cNorm  = colors.Normalize(vmin=0, vmax=l)
        scalarMap = cm.ScalarMappable(norm=cNorm, cmap=colorMap,)
        return scalarMap;

SImulation Plots
===
Local Simulation
---

In [ ]:
# 
vv.closeAll()
folderPath = "../LocalFolder/ProcessMPI_0/SimDataRECORDMPI_0/"
filePathRegex = ".*TopologyInfo_(\d*).xml"
index=range(1,26);
p = PlotTopologyMPI(folderPath,filePathRegex,index);

# for i, t in enumerate(p.topoInfos.values()):
#      fname=os.path.join(folderPath,"PointCloud_%i.txt" % i);
#      np.savetxt(fname, t.points, fmt='%f', delimiter=' ', newline='\n', header='', footer='', comments='# ');

p.plot(index=index,ax=None,oneWindow=True,plotPoints=True,plotSubDivs=False);

Avalanche1M-DriftC-2
---

In [ ]:
#
plt.close('all')
folderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-DriftC-2/second/ProcessMPI_0/SimDataRECORDMPI_0"
filePathRegex = ".*TopologyInfo_(\d*).xml"
index=range(1,2);
p = PlotTopologyMPI(folderPath,filePathRegex,index);


# for i, t in enumerate(p.topoInfos.values()):
#      fname=os.path.join(folderPath,"PointCloud_%i.txt" % i);
#      np.savetxt(fname, t.points, fmt='%f', delimiter=' ', newline='\n', header='', footer='', comments='# ');


p.plot(index,ax=None,oneWindow=True,plotPoints=True);

Avalanche1M-FillUp1
---

In [ ]:
#
plt.close('all')
folderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp/1/ProcessMPI_0/SimDataRECORDMPI_0"
filePathRegex = ".*TopologyInfo_(\d*).xml"
index=range(1,5);
p = PlotTopologyMPI(folderPath,filePathRegex,index);
p.plot(index,ax=None,oneWindow=True,plotPoints=True,plotSubDivs=False);

In [ ]:
#
localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp/1/"

simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI);

p.plot(oneWindow=False)

Avalanche1M-FillUp3
---

In [ ]:
#
localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp3/1/"

simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI);

p.plot(oneWindow=False)

Avalanche1M-FillUp4
---

In [ ]:
#
localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp4/1/"

simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI);

p.plot(oneWindow=False)

In [ ]:
#
plt.close('all')
for key,value in p.simDataList['Average'].items():
    print(key)
    fd = np.array(value);
    p.simDataList['Average'][key] = np.average(fd,axis=0);
    
plt.plot(p.simDataList['Average']['MaxOverlap'])

In [ ]:
#
plt.close('all')
localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp4/2/"

simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI);

p.plot(oneWindow=False)

In [ ]:
#
plt.close('all')
localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp5/1/"

simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI);

p.plot(oneWindow=False)

In [ ]:
#
bodyRange = [0,50]
timeRange = [0,5]

#For single file which contains multiple bodies
#dynState = ReadInSimFile("./SimFiles/SimDataRECORDGUI_0/SimulationState.sim",[0,10],[0,10],False)
#For multiple file which contains 1 body!
#dynState = ReadInSimFileMulti("./SimFiles/SimDataRECORD_0/", [0,10] , [0,10], "SimDataBody-1-(\d*)\.sim" ,True)
#print(dynState)

folderNumber= 0
localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp5/1/"
simFileRelPath = "./SimState.sim"
filePath = os.path.join(localFolderPath , simFileRelPath);
#filePath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-DriftC/first/SimStateRenderTest.sim"

dynState = SimFileReader.ReadInSimFile( filePath ,timeRange,bodyRange,False)

plotData(dynState)
res = plotData3D(dynState);


In [ ]:
#
plt.close('all')

folderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp/1/ProcessMPI_0/SimDataRECORDMPI_0"
filePathRegex = ".*TopologyInfo_(\d*).xml"
index=range(1,5);
p = PlotTopologyMPI(folderPath,filePathRegex,index);
p.plot(index,ax=None,oneWindow=True,plotPoints=True,plotSubDivs=False);

AvalancheFillUp5
---

In [ ]:
#
plt.close('all')
localFolderPath = "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-FillUp5/1/"

simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI);

p.plot(oneWindow=False)

AvalancheRunDownTrees
---

SimData Plots

In [ ]:
#
plt.close('all')
localFolderPath = "/media/zfmgpu/Data1/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-RunDownTrees/1/"
simDataRelPath = "./SimDataRECORDMPI_0/SimData.dat"
regExProcessFolder=".*ProcessMPI_(\d*).*"
simDataPathGUI=""
p = PlotSimDataMPI(localFolderPath,simDataRelPath,regExProcessFolder,simDataPathGUI);
p.plot(oneWindow=False)

Topology Plots

In [ ]:
# make symlinks
plt.close('all')
localFolderPath = "/media/zfmgpu/Data1/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-RunDownTrees";


In [ ]:
makeSymlinks(localFolderPath+ "/*/ProcessMPI_0/SimDataRECORDMPI_0/TopologyInfo_*.xml", 
                    r".*Trees/(\d)*/.*_(\d)*.*", 
                    r"TopologyInfo_\1\2",
                    "/media/zfmgpu/Data1/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-RunDownTrees/AllTopologies/",
                     reEnumerateStart=1)

In [ ]:
#
folderPath =  localFolderPath +  "/AllTopologies/"
#folderPath = "/home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/SimulationFramework/Simulations/LocalFolder/ProcessMPI_0/SimDataRECORDMPI_0"
filePathRegex = ".*TopologyInfo_\d*-(\d*).xml"
index=range(1,19);
p = PlotTopologyMPI(folderPath,filePathRegex,index);
p.plot(index,ax=None,oneWindow=True,plotPoints=True,plotSubDivs=False);

In [ ]:

folderPath = "../"
filePathRegex = ".*TopologyInfo_(\d*).xml"
index=range(1,2);
p = PlotTopologyMPI(folderPath,filePathRegex,index);

# for i, t in enumerate(p.topoInfos.values()):
#      fname=os.path.join(folderPath,"PointCloud_%i.txt" % i);
#      np.savetxt(fname, t.points, fmt='%f', delimiter=' ', newline='\n', header='', footer='', comments='# ');

p.plot(index=index,ax=None,oneWindow=False,plotPoints=False,plotSubDivs=False);

In [ ]:
vv.closeAll()
fig = vv.figure();
fig.title = "TopoInfo " + "1";
ax = vv.gca();
ax.cameraType = '3d'
ax.daspectAuto = False
vv.xlabel("x");
vv.ylabel("y");
vv.zlabel("z");


# for i in range(0,1):
# #     v =np.random.uniform(0,1,(3,));
#     plotCubeMesh(ax)

plotCubeMesh(ax, minP=np.array([1,1,1]), maxP=np.array([3,2,2]),rotQuat= axangle2quat(np.array([1,1,0]),0.0001) )    
app = vv.use();
app.Run();


In [ ]:

# makeSymlinkSequence("/media/zfmgpu/Data1/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-RunDownTrees"+
#                     "/*/ProcessMPI_0/SimDataRECORDMPI_0/TopologyInfo_*.xml", 
#                     r".*Trees/(\d)*/.*_(\d)*.*", 
#                     r"TopologyInfo_\1-\2",
#                     "/media/zfmgpu/Data1/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche1M-RunDownTrees/AllTopologies/")

import re,os;
makeLinks("/media/zfmgpu/Data3/Renderings/cluster/work/scr1/nuetzig/MyRenderJob.0" +
                     "/*/output/renders/Frame*",
                     r".*?Frame-(\d{4})\.exr",
                     r"Frame-{0:03d}.exr",
                     "/media/zfmgpu/Data3/Renderings/cluster/work/scr1/nuetzig/MyRenderJob.0/AllFrames/",
             hardLink = True, counters = [(1,lambda x: os.path.basename(x))]
                    )

In [ ]:
a=['/media/zfmgpu/Data3/Renderings/cluster/work/scr1/nuetzig/MyRenderJob.0/Process_1/output/renders/Frame-0262.exr', '/media/zfmgpu/Data3/Renderings/cluster/work/scr1/nuetzig/MyRenderJob.0/Process_10/output/renders/Frame-0271.exr', '/media/zfmgpu/Data3/Renderings/cluster/work/scr1/nuetzig/MyRenderJob.0/Process_100/output/renders/Frame-0182.exr']
a.sort(key = lambda file: (os.path.dirname(file).replace("/",""), os.path.basename(file)))
print(a)

In [ ]:
sorted([(4,1,1),(2,1,3),(1,2,3),(1,1,2),(1,1,0)])